In [25]:
import pandas as pd
import yfinance as yf
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

oil_gas_tickers = ['XOM', 'CVX', 'BP', 'COP', 'SLB', 'HAL', 'BKR']
index_ticker = '^GSPC'  # S&P 500 index

df_list = []

for ticker in oil_gas_tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=oil_gas_tickers, names=['Ticker', 'Date'])
print(df.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
XOM    2023-04-19  116.080002  116.900002  115.769997  116.570000  115.567177   
       2023-04-20  115.129997  115.680000  113.230003  115.639999  114.645180   
       2023-04-21  116.489998  116.879997  115.260002  116.010002  115.012001   
       2023-04-24  115.919998  118.889999  115.879997  118.199997  117.183151   
       2023-04-25  117.550003  

In [26]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,62.000000,62.000000,62.000000,62.000000,62.000000,6.200000e+01
mean,29.794193,30.261210,29.460968,29.912258,29.828245,6.998783e+06
std,1.991095,2.038932,2.037492,2.086221,2.147041,2.273947e+06
min,27.040001,27.530001,26.809999,27.219999,27.035294,2.951200e+06
25%,28.282500,28.552501,27.725000,28.182500,27.991262,5.499800e+06
50%,29.640000,30.299999,29.250000,29.780001,29.780001,6.828950e+06
75%,30.612500,31.085001,30.280001,30.627500,30.627500,7.941925e+06
max,34.509998,35.294998,34.470001,34.860001,34.860001,1.791730e+07


In [27]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0    XOM 2023-04-19  116.080002  116.900002  115.769997  116.570000   
1    XOM 2023-04-20  115.129997  115.680000  113.230003  115.639999   
2    XOM 2023-04-21  116.489998  116.879997  115.260002  116.010002   
3    XOM 2023-04-24  115.919998  118.889999  115.879997  118.199997   
4    XOM 2023-04-25  117.550003  117.739998  115.629997  116.519997   

    Adj Close    Volume  
0  115.567177  10682600  
1  114.645180  17038700  
2  115.012001  14315300  
3  117.183151  15831300  
4  115.517609  12179800  


In [28]:
import plotly.express as px
fig = px.line(df, x='Date', y='Close', color='Ticker', title="Stock Market Performance for the Last 3 Months")
fig.show()

In [29]:
fig = px.area(df, x='Date', y='Close', color='Ticker',facet_col='Ticker', 
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Oil & Gas Companies')
fig.show()

# Moving Average

### Bullish Signal
MA10 crosses above the MA20, the stock price will continue to rise.

### Bearish Signal
MA10 crosses below the MA20, the stock price will continue falling.

In [30]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'], title=f"{ticker} Moving Averages")
    fig.show()

# Volatility

In [31]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', color='Ticker', title='Volatility of All Companies')
fig.show()

In [32]:
# Retrieve all available stock symbols
all_stock_symbols = yf.Tickers(' '.join(oil_gas_tickers + [index_ticker]))

# Download historical stock data for all available symbols
combined_data = all_stock_symbols.history(start=start_date, end=end_date)

[*********************100%***********************]  8 of 8 completed


In [34]:
# Select the 'Close' prices for oil and gas stocks and other stocks
selected_data = combined_data['Close'][oil_gas_tickers + [index_ticker]]

correlation_matrix = selected_data.corr()

# Find the strongest correlations with oil and gas stocks
strongest_correlations = correlation_matrix[oil_gas_tickers].drop(oil_gas_tickers)

# Sort the correlations in descending order
strongest_correlations = strongest_correlations.stack().sort_values(ascending=False)

# Print the stocks with strong correlation to oil and gas stocks
print("Stocks with Strong Correlation to Oil and Gas Stocks:")
print(strongest_correlations)

Stocks with Strong Correlation to Oil and Gas Stocks:
^GSPC  BKR    0.871217
       COP    0.726471
       HAL    0.718635
       SLB    0.653214
       CVX   -0.321779
       BP    -0.489089
       XOM   -0.500222
dtype: float64
